# ReJSON: a JSON data type

ReJSON is a Redis module that implements [ECMA-404 The JSON Data Interchange Standard](http://json.org/) as a native data type. It allows storing, updating and fetching JSON values from Redis keys (documents).

Primary features:

* Full support of the JSON standard
* [JSONPath](http://goessner.net/articles/JsonPath/)-like syntax for selecting elements inside documents
* Documents are stored as binary data in a tree structure, allowing fast access to sub-elements
* Typed atomic operations for all JSON values types

The ReJSON website is at [http://rejson.io](http://rejson.io).

ReJSON is open source, the code is available at [https://github.com/redislabsmodules/rejson](https://github.com/redislabsmodules/rejson).

## What is JSON

> **JSON** (JavaScript Object Notation) is a lightweight data-interchange format. It is easy for humans to read and write. It is easy for machines to parse and generate. It is based on a subset of the [JavaScript Programming Language](http://javascript.crockford.com/), [Standard ECMA-262 3rd Edition - December 1999](http://www.ecma-international.org/publications/files/ecma-st/ECMA-262.pdf). JSON is a text format that is completely language independent but uses conventions that are familiar to programmers of the C-family of languages, including C, C++, C#, Java, JavaScript, Perl, Python, and many others. These properties make JSON an ideal data-interchange language.
>
> JSON is built on two structures:
>
> - A collection of name/value pairs. In various languages, this is realized as an *object*, record, struct, dictionary, hash table, keyed list, or associative array.
> - An ordered list of values. In most languages, this is realized as an *array*, vector, list, or sequence.
>
> These are universal data structures. Virtually all modern programming languages support them in one form or another. It makes sense that a data format that is interchangeable with programming languages also be based on these structures.

Source: [http://json.org/](http://json.org/)

## JSON and core Redis

To manage JSON data using Redis' core data structures, the first decision that needs to be made is whether the values are stored in the database in their serialized form or not.

### Serialized JSON

A serialized JSON value is a string, which happens to be one of Redis' core data structure. Any JSON-encoded data (up to 0.5GB) can be stored as is in the database and at top performance.

In [ ]:
import redis

# example connection parameters 
config = {
    "host": "redis",
    "port": 6379
}

r = redis.StrictRedis(**config)
import json

mydoc = {
    'foo': 'bar',
    'ans': 42
}

# Setting JSON in a Redis String is easy...
r.set('mydoc', json.dumps(mydoc))

# ... and so is getting it back
print (json.loads(r.get('mydoc')))

The main characteristic of this approach is that it only allows access to the value as a whole - the entire JSON payload is either read or written in every operation, which may be less than desirable in some cases.

This effect can be mitigated by employing Redis' Lua engine and executing server-side scripts that use the built-in [`cjson` library](https://redis.io/commands/eval#cjson). The following example is a script that returns a specific sub-element by path:

In [ ]:
# Lua scripting is like having super powers
jsonGetPath = r.register_script("""
-- Get the value and deserialize it
local js = redis.call('GET', KEYS[1])
local v = cjson.decode(js)

-- Parse the path
local r = v
while #ARGV > 0 do
  local token = table.remove(ARGV,1)
  local n = tonumber(token)
  if n then
    r = r[n+1]
  else
    r = r[token]
  end
end

-- Serialize the reply back to JSON
local rj = cjson.encode(r)
return rj
""")

print (jsonGetPath(keys=['mydoc'], args=['foo']))

Furthermore, when using Lua scripts it is possible to replace the JSON format that's used internally with the more compact and performant [MessagePack](http://msgpack.org) through the built-in [`cmsgpack` library](https://redis.io/commands/eval#cmsgpack) - for more scripting examples [refer here](https://github.com/RedisLabsModules/rejson/tree/master/benchmarks/lua).

Combining raw JSON storage and Lua in Redis is a decent solution, but the constant encoding/decoding of values can introduce a performance penalty as the values get larger.

### Deserialized JSON

The opposite approach to storing raw serialized JSON values, is to deserialize them and store them that way. Consider the following example that uses the core Redis Hash to store a document:

In [ ]:
# Lets start over
r.delete('mydoc')

# Use a Redis Hash to store the document
r.hmset('mydoc', mydoc)

# Access a specific field
print (r.hget('mydoc', 'foo'))

While the above is more than enough for "flat" objects, this approach quickly gets complex (not to say totally broken) once the objects become nested. You can't store an object in an object, have arrays (Redis Lists) in them and so forth... Furthermore, this approach still requires the application code to encode Redis' reply to JSON (that is assuming that we are dealing with JSON objects).

### Trade offs

The following table summarizes some of the main trade offs between the approaches for storing JSON using Redis' core capabilities:

|                                    | Serialized               | Serialized + Lua                     | Deserialized    |
| ---------------------------------- | ------------------------ | ------------------------------------ | --------------- |
| GET/SET entire document complexity | O(1)                     | O(1)                                 | O(N) and beyond |
| GET/SET sub-element complexity     | N/A                      | O(document complexity)               | O(1)            |
| Supports nesting, arrays, ...      | Yes                      | Yes                                  | No              |
| Atomic updates                     | Yes (with transactions)  | Yes                                  | Yes             |
| Accepts/returns valid JSON         | Yes                      | Yes                                  | No              |
| Memory overheads                   | JSON is somewhat verbose | Same, although MessagePack is better | Hash overheads  |

## Why ReJSON

As shown above, Redis' core data structures can be used for managing JSON values, but each approach has its own trade offs. One could invest the effort to combine the approaches to provide a robust solution, but at the cost of doubling the memory requirement at the very least. On top of that, one could argue that the developer's experience of mixing JSON and Redis is less than ideal.

ReJSON was born out of those needs, and provides an API that is consistent and familiar both to Redis as well as to JSON users.

## Training prelude

The following sections in this notebook show how to use ReJSON with Python. While ReJSON can be used with any Redis client, it has several language-specific implementations that provide an interface to the module's API. The project's website has the [full list](http://rejson.io/#client-libraries). 

Python's ReJSON client, [rejson-py](https://github.com/RedisLabs/rejson-py/), is installable by:

<code><pre>
$ pip install rejson
</pre></code>

rejson-py extends [redis-py](https://github.com/andymccurdy/redis-py)'s interface with ReJSON's API, so it can be used like a regular client but also performs on-the-fly serialization/deserialization of objects to/from JSON via Python's [`json`](https://docs.python.org/2/library/json.html) library.

## Creating the client

To begin using ReJSON with its Python client, the first thing that's needed is to create the client. The client is created in the same manner as a regular client:

In [ ]:
from rejson import Client, Path

rj = Client(**config)

## Setting and getting JSON scalar values

The first basic operation that ReJSON provides is setting JSON values in documents. A (Re)JSON document is stored under a Redis key and can be set with the [`JSON.SET`](http://rejson.io/commands/#jsonset) command:

<code><pre>
JSON.SET <key> <path> <json> [NX|XX]
</pre></code>

The name of the key is given by the `<key>` argument. If the key does not exist, it is created. If the key exists and is a JSON document, it is updated. If the key exists but isn't a ReJSON value, an error is thrown.

`<path>` is a resolvable location in the document. ReJSON uses a JSONPath-like syntax that will be detailed below, where a single period (`.`) character represents the document's root. Every document has exactly one root node, and new documents must be created at the root.

The optional `NX` (not exists) and `XX` (already exists) flags have the same meaning as the the standard Redis existential modifiers.

The JSON value is given by the `<json>` argument - a valid, serialized JSON string is expected. Wh en `JSON.SET` is called, the serialized input JSON is syntactically validated, parsed and the corresponding tree data structures representing it is generated:

![ReJSON data structure](/static/img/rejson-data-structure.png)

The second basic operation is getting values, which is exactly what [`JSON.GET`](http://rejson.io/commands/#jsonget) does:

<code><pre>
JSON.GET <key>
         [INDENT indentation-string]
         [NEWLINE line-break-string]
         [SPACE space-string]
         [path ...]
</pre></code>

Scalars are JSON's non-nested values - null, Boolean true and false, numbers and strings. This example shows setting and getting a JSON string in Python using the client's [`jsonset()`](https://github.com/RedisLabs/rejson-py/blob/master/API.md#jsonset) and [`jsonget()`](https://github.com/RedisLabs/rejson-py/blob/master/API.md#jsonget) methods:

In [ ]:
# Set the ReJSON key `don't` to the JSON string `panic!`, get the value back and print it
key = 'don\'t'
val = 'panic'
rj.jsonset(key, Path.rootPath(), val)
rep = rj.jsonget('don\'t')
print '{} {}'.format(key, rep)

## Exercise: getting and setting values

1. Set and get the number 42 to and from the key named *"Answer to the Ultimate Question of Life, the Universe, and Everything"*

   ```python
   question_key = 'Answer to the Ultimate Question of Life, the Universe, and Everything'
   question_answer = 42
   # Your answer here
   ```

2. Update the answer to the question from the first exercise to the string *"six by nine"*

   ```python
   what_is_42 = 'six by nine'
   # Your answer here
   ```

3. Set and get nothing (`None`) from the empty string key (*""*)

   ```python
   empty = ''
   #
   ```

4. Change the following and create a paradox:

   ```python
   rj.jsonset('My value is true', True)
   ```

## Down the rabbit hole: nested values

On top of scalars, there are JSON's two types of container structures: objects (key-value dictionaries) and arrays. A JSON container can contain zero or more JSON values, including other containers.

Naturally, ReJSON supports these as well:

In [ ]:
ix = {
    'name': 'Ix',
    'alias': 'Ford Prefect',
    'human': False,
    'gender': None,
    'age': 200.5,
    'heads': 1,
    'origin': 'Betelgeuse 5',
    'job': 'Researcher, The Hitchhiker\'s Guide to the Galaxy',
    'inventory': {
          'electronic thumb': 1,
          'wholly remarkable book': 1,
          'towel': 1
    },
    'not in ventory': [
          'toothbrush', 'washcloth', 'soap', 'tin of biscuits', 'flask', 'compass',
          'map', 'ball of string', 'gnat spray', 'wet-weather gear', 'space suit', 'etc.'
    ]
}
rj.jsonset('ix', Path.rootPath(), ix)

rejson-py's `jsonset()` (and `jsonget()`) performs on-the-fly serialization (and deserialization) from Python's data types to (and from) JSON using the default Pythonic `json` library. This allows getting back the objects (and arrays)  as easily as:

In [ ]:
ford = rj.jsonget('ix')
print '{}, a.k.a. {}, is a {} years-old {} from {} who {} a towel'.format(
    ford['name'], ford['alias'], ford['age'],
    'human' if ford['human'] else 'alien', ford['origin'],
    'carries' if int(ford['inventory']['towel']) > 0 else 'doesn\'t have'
)

## Sub-elements selection using paths

The example above requested (and got back) the entire JSON document with the command `rj.jsonget('ix')`. When left unspecified, `jsonget()`'s path defaults to the document's root, so the entire JSON value (an object) is traversed, serialized and returned. While getting (and setting) values in whole is very useful, sometimes a more granular access to the document's sub-elements is needed.

A path describes the route from the root to a node in the tree. Paths always begin at the root of a ReJSON value. The root is denoted by the period character (`.`). For paths referencing the root's children, prefixing the path with the root is optional. Examples:

Dotted- and square-bracketed, single-or-double-quoted-child notation are both supported for object keys, so the following paths all refer to _bar_, child of _foo_ under the root:

* `.foo.bar`
* `foo["bar"]`
* `['foo']["bar"]`

Array elements are accessed by their index enclosed by a pair of square brackets. Like with Python and Redis, the index is 0-based, with 0 being the first element of the array, 1 being the next element and so on. These offsets can also be negative numbers indicating indices starting at the end of the array. For example, -1 is the last element in the array, -2 the penultimate, and so on.

### A note about JSON key names and path compatibility

By definition a JSON key can be any valid JSON String. Paths, on the other hand, are traditionally based on JavaScript's (and those on Java's in turn) variable naming conventions. Therefore, while it is possible to have ReJSON store objects containing arbitrary key names, accessing these keys via a path will only be possible if they respect these naming syntax rules:

1. Names must begin with a letter, a dollar (`$`) or an underscore (`_`) character
2. Names can contain letters, digits, dollar signs and underscores
3. Names are sensitive, case-wise

## Exercise: using paths

1. Replace the root path in the below so the code will print the number of towels that Ford has with it:

   ```python
   print rj.jsonget('ix', Path('.'))
   ```

2. Uncomment the next line, and use the correct path so Ford will also not have a toothpick instead of the "etc." placeholder:

   ```python
   # rj.jsonset('ix', Path('.somewhere.in.the.tory[00]...'), 'toothpick')
   ```

## Getting multiples

In some cases the information that we're after is stored under multiple paths inside the same document. While it is perfectly possible to call `jsonget()` multiple times (and if so, preferably in a pipeline), you can also invoke it with a variable number of path arguments and it will return a JSON object containing all the values under their respective named-by-path keys. For example:

In [ ]:
print rj.jsonget('ix', Path('.alias'), Path('age'), Path('inventory.towel'))

In other cases, we have multiple documents (keys) and we're interested in reading the same path from all of them. That is where the [`JSON.MGET`](http://rejson.io/commands/#jsonmget) comes in handy, available via the client's [`jsonmget()`](http://rejson.io/commands/#jsonmget) method. It accepts a single path and one or more key names, for example:

In [ ]:
# First create another alien
bz = {
    'name': 'Zaphod Beeblebrox',
    'alias': 'Big Z',
    'human': False,
    'heads': 2,
    'arms': 3,
    'job': 'President of the Galaxy, Former',
    'inventory': {
        'spaceship with an infinite improbablity drive': 1
    }
}

rj.jsonset('bz', Path.rootPath(), bz)

aliens = ['ix', 'bz']
print 'Aliens: {}'.format(aliens)
print 'Heads: {}'.format(rj.jsonmget(Path('.heads'), *aliens))
print 'Arms: {}'.format(rj.jsonmget(Path('.arms'), *aliens))

Note that `jsonmget()`'s returns a list (or what is technically a Redis bulk array), in which each element corresponds to the path's value in the target keys. The order of values is the order that the keys are provided at. A `None` (i.e. a Redis null) is returned in the reply for paths/keys that do not exist.

## Deleting values and documents

A ReJSON document is just a key in Redis' keyspace, so deleting an entire document can be achieved with Redis' [`DEL`](https://redis.io/commands/del) command (and `delete()` when using the redis-py/rejson-py client).

It is possible to delete a sub-element inside a JSON value with the [`JSON.DEL`](http://rejson.io/commands/#jsondel):

<code><pre>
JSON.DEL <key> <path>
</pre></code>

Deleting the a document's root node is equivalent to deleting the key itself. The command's respective rejson-py counterpart - [`jsondel()`](https://github.com/RedisLabs/rejson-py/blob/master/API.md#jsondel) - is invoked in a straightforward fashion:

In [ ]:
rj.jsonset('solar_system', Path.rootPath(),
           ['sun', 
            'mercury', 'venus', 'earth', 'mars',
            'jupiter', 'saturn', 'uranus', 'neptune',
            'pluto']
)

# Make way for a new hyperspace bypass
rj.jsondel('solar_system', Path('[3]'))
print 'The Solar System, post Vogon fleet visit: {}'.format(rj.jsonget('solar_system'))

## Types of values and checking for existence

The type of a JSON value can be begotten with a call to [`JSON.TYPE`](http://rejson.io/commands/#jsontype)/[`jsontype()`](https://github.com/RedisLabs/rejson-py/blob/master/API.md#jsontype):

In [ ]:
print rj.jsontype('ix', Path('.gender'))
print rj.jsontype('ix', Path('human'))
print rj.jsontype('ix', Path('age'))
print rj.jsontype('ix', Path('name'))
print rj.jsontype('ix')
print rj.jsontype('solar_system')

`jsontype()` is also handy for checking whether a value exists, as it returns `None` when it doesn't.

## Operations on numbers

ReJSON provides two operations on JSON number values: addition and multiplication, or [`JSON.NUMINCRBY`](http://rejson.io/commands/#jsonnumincrby) and [`JSON.NUMMULTBY`](http://rejson.io/commands/#jsonnummultby), respectively. The syntax for is identical for both,  consisting of a key name, a path to a number value and a number argument to perform the operation with:

<code><pre>
JSON.NUMINCRBY <key> <path> <number>
JSON.NUMMULTBY <key> <path> <number>
</pre></code>

The following example demonstrates using number operations in a pipelined transaction:

In [ ]:
with rj.pipeline() as p:
    p.jsonset('number', Path.rootPath(), 6379)
    p.jsonnumincrby('number', Path.rootPath(), -79)
    p.jsonnummultby('number', Path.rootPath(), 0.5/75)
    number = p.jsonget('number')
    p.execute()
print 'And the number is... (drumroll)... {}!'.format(number)

## String operations

Strings also have their own typed commands: [`JSON.STRAPPEND`](http://rejson.io/commands/#jsonstrappend) appends a string to a JSON string value, while [`JSON.STRLEN`](http://rejson.io/commands/#jsonstrlen) reports a string's length:

In [ ]:
ad = {
    'name': 'Arthur'
}

rj.jsonset('ad', Path.rootPath(), ad)
print 'Length of name, before: {}'.format(rj.jsonstrlen('ad', Path('name')))
print 'Length of name, after: {}'.format(rj.jsonstrappend('ad', ' Dent', Path('name')))
print 'Name, after: {}'.format(rj.jsonget('ad', Path('name')))

## Object commands

ReJSON can report an object's keys and key count with [`JSON.OBJKEYS`](http://rejson.io/commands/#jsonobjkeys) and [`JSON.OBJLEN`](http://rejson.io/commands/#jsonobjlen), respectively:

In [ ]:
print 'Ix is in posession of: {}'.format(rj.jsonobjkeys('ix', Path('inventory')))
print 'The number of item classes that Ix has is {}'.format(rj.jsonobjlen('ix', Path('inventory')))

## Manipulating arrays

While ReJSON can report an array's length with the [`JSON.ARRLEN`](http://rejson.io/commands/#jsonarrlen) command, there are several additional operations that it supports. [`JSON.ARRAPPEND`](http://rejson.io/commands/#jsonarrappend) adds one or more values at the end of an array, whereas [`JSON.ARRINSERT`](http://rejson.io/commands/#jsonarrinsert) does the same but at any arbitrary offset in the array:

In [ ]:
rj.jsonset('array', Path.rootPath(), [0])
rj.jsonarrappend('array', Path.rootPath(), 1, 2)
rj.jsonarrinsert('array', Path.rootPath(), 0, -2, -1)
print rj.jsonget('array')

To search for a scalar value in the array, use [`JSON.ARRINDEX`](http://rejson.io/commands/#jsonarrindex):

In [ ]:
where_is_0 = rj.jsonarrindex('array', Path.rootPath(), 0)
print '0\'s index: {}'.format(where_is_0)

Arrays also can be trimmed from both ends to a specified range of elements using the [`JSON.ARRTRIM`](http://rejson.io/commands/#jsonarrtrim) command:

In [ ]:
# Trim the negative numbers
rj.jsonarrtrim('array', Path.rootPath(), where_is_0, -1)
print rj.jsonget('array')

Lastly, elements in the array can be popped (i.e. deleted and returned to the caller) by calling [`JSON.ARRPOP`](http://rejson.io/commands/#jsonarrpop):

In [ ]:
print 'Popped {}'.format(rj.jsonarrpop('array'))
print 'Popped {}'.format(rj.jsonarrpop('array'))
print 'Popped {}'.format(rj.jsonarrpop('array'))
print 'Popped {}'.format(rj.jsonarrpop('array'))